In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./data/02.데이터수집_01샘플링_고객데이터샘플링_고객정보_9999명.csv')
df.head()

,성명,생년월일,성별
0,Krystal,19911009,남
1,Mary,19860513,남
2,Kerri,19840914,여
3,John,20020708,여
4,Sara,19980926,여


In [5]:
df1 = pd.read_csv('./data/02.데이터수집_01샘플링_고객데이터샘플링_고객성향DB_9999명.csv')
df1.head()

,피부톤,피부타입,피부고민,선호스타일
0,가을웜톤,건성,트러블,데일리
1,여름쿨톤,중성,블랙헤드/모공,유니크(힙)
2,봄웜톤,지성,블랙헤드/모공,섹시
3,가을웜톤,민감성,주름/탄력,섹시
4,봄웜톤,복합성,트러블,청순


## 고객정보와 고객성향정보는 중간에 수기로 엑셀저장 작업

#### 1. 고객 기본정보 DB 넣기

In [3]:
excel_file ='./data/02.데이터수집_03DB저장_수집정보전체_고객 기본정보 DB (샘플).xlsx'
df = pd.read_excel(excel_file)
df.head()

,userID,성함,주소,이메일,연락처,아이디,비밀번호,생년월일(6자리),성별(남/여)
0,M0001,공유,NaN,NaN,NaN,NaN,NaN,730905,남
1,F0002,송혜교,NaN,NaN,NaN,NaN,NaN,800520,여
2,F0003,이소연,NaN,NaN,NaN,NaN,NaN,910909,여
3,F0004,하양이,NaN,NaN,NaN,NaN,NaN,1111,여
4,F0005,괄괄이,NaN,NaN,NaN,NaN,NaN,20201,여


In [4]:
# 생년월일 0 패딩처리
df['생년월일(6자리)'] = df['생년월일(6자리)'].map('{:06d}'.format)


In [5]:
df.head()

,userID,성함,주소,이메일,연락처,아이디,비밀번호,생년월일(6자리),성별(남/여)
0,M0001,공유,NaN,NaN,NaN,NaN,NaN,730905,남
1,F0002,송혜교,NaN,NaN,NaN,NaN,NaN,800520,여
2,F0003,이소연,NaN,NaN,NaN,NaN,NaN,910909,여
3,F0004,하양이,NaN,NaN,NaN,NaN,NaN,001111,여
4,F0005,괄괄이,NaN,NaN,NaN,NaN,NaN,020201,여


In [6]:
df.columns

Index(['userID', '성함', '주소', '이메일', '연락처', '아이디', '비밀번호', '생년월일(6자리)',
       '성별(남/여)'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userID     9999 non-null   object 
 1   성함         9999 non-null   object 
 2   주소         1 non-null      object 
 3   이메일        0 non-null      float64
 4   연락처        0 non-null      float64
 5   아이디        0 non-null      float64
 6   비밀번호       0 non-null      float64
 7   생년월일(6자리)  9999 non-null   object 
 8   성별(남/여)    9999 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.2+ KB


In [8]:
# 열 이름 적절하게 변경
df.rename(columns={'성함':'name', '주소':'address', '이메일':'email', '연락처':'phonenum','아이디':'nickname','비밀번호':'password', '생년월일(6자리)':'birth','성별(남/여)':'sex'},inplace=True)

In [14]:
df.head()

,userID,name,address,email,phonenum,nickname,password,birth,sex
0,M0001,공유,-,-,-,-,-,730905,남
1,F0002,송혜교,-,-,-,-,-,800520,여
2,F0003,이소연,-,-,-,-,-,910909,여
3,F0004,하양이,-,-,-,-,-,001111,여
4,F0005,괄괄이,-,-,-,-,-,020201,여


In [10]:
# NaN값 '-'로 대체
df.fillna('-',inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userID    9999 non-null   object
 1   name      9999 non-null   object
 2   address   9999 non-null   object
 3   email     9999 non-null   object
 4   phonenum  9999 non-null   object
 5   nickname  9999 non-null   object
 6   password  9999 non-null   object
 7   birth     9999 non-null   object
 8   sex       9999 non-null   object
dtypes: object(9)
memory usage: 703.2+ KB


In [12]:
## DB연결후, 저장

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='1111',\
    db = 'beaulol', charset='utf8')
cur= conn.cursor()

cur.execute(""" 
            create table if not exists customer_info ( 
            userID VARCHAR(10) NOT NULL,
            name VARCHAR(30) NOT NULL, 
            address VARCHAR(200), 
            email VARCHAR(50), 
            phonenum  VARCHAR(30),
            nickname VARCHAR(20), 
            password VARCHAR(40), 
            birth VARCHAR(6) NOT NULL,
            sex VARCHAR(1) NOT NULL,
            PRIMARY KEY(userID)
            )
            """)

for idx,row in df.iterrows():
    try:
        sql = "INSERT INTO customer_info(userID,name,address,email,phonenum,nickname,password,birth,\
        sex) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        cur.execute(sql, (row['userID'],row['name'],row['address'],row['email'], row['phonenum'],\
            row['nickname'], row['password'],row['birth'],row['sex']))
        
    except Exception as e:
        print("Error:", e)
        continue

conn.commit()
conn.close()

Error: (1062, "Duplicate entry 'M0001' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0002' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0003' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0004' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0005' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0006' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'M0007' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0008' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0009' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0010' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0011' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0012' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0013' for key 'customer_info.PRIMARY'")
Error: (1062, "Duplicate entry 'F0014' for key 'cus

#### 2. 고객 성향정보 DB넣기

In [15]:
excel_file2 ='./data/02.데이터수집_03DB저장_수집정보전체_고객 성향정보 DB (샘플).xlsx'
df2 = pd.read_excel(excel_file2)
df2.head()

,userID,피부톤,피부타입,피부고민,선호 스타일
0,NaN,쿨톤,지성,잡티(-미백-홍조-다크서클),청순
1,NaN,웜톤,건성,주름(-탄력),데일리
2,NaN,봄웜톤,복합성,각질,유니크(힙)
3,NaN,여름쿨톤,민감성,트러블,섹시
4,NaN,가을웜톤,중성,블랙헤드(-모공),러블리


In [17]:
len(df2['userID'])

10005

In [18]:
# 위에 설명행 제거
df2.drop(range(6),inplace=True)

In [19]:
len(df2['userID'])

9999

In [20]:
# # userID 데이터 값 추가
# df2['userID'] = ['M0001', 'F0002', 'F0003', 'F0004', 'F0005', 'F0006', 'M0007', 'F0008', 'F0009', 'F0010',
#                  'F0011', 'F0012', 'F0013', 'F0014', 'F0015', 'M0016', 'M0017', 'M0018', 'M0019', 'M0020',
#                  'F0021', 'F0022', 'F0023', 'F0024', 'F0025', 'F0026', 'F0027', 'F0028', 'F0029', 'F0030']

In [21]:
# 기초그룹의 NaN값 '-'로 변경
# df2['기초 그룹'].fillna('-',inplace=True)

In [22]:
# # 필요없는 열 제거
# df2.drop(columns=['Unnamed: 6','Unnamed: 7'],inplace=True)
# 열 이름에서 공백 제거
df2.rename(columns={'피부톤':'user_tone', '피부타입':'user_skintype', '피부고민':'user_trouble', '선호 스타일':'user_style'},inplace=True)

In [23]:
# 피부고민 열에서 하이픈'-'을 슬래시'/'로 변경
df2['user_trouble'] = df2['user_trouble'].str.replace('-','/')

# 괄호 제거
df2['user_trouble'] = df2['user_trouble'].str.replace('(','').str.replace(')','')

In [24]:
df2.head()

,userID,user_tone,user_skintype,user_trouble,user_style
6,M0001,쿨톤,복합성,블랙헤드/모공,유니크(힙)
7,F0002,가을웜톤,건성,민감성/아토피,데일리
8,F0003,겨울쿨톤,복합성,주름/탄력,유니크(힙)
9,F0004,봄웜톤,중성,잡티/미백/홍조/다크서클,러블리
10,F0005,겨울쿨톤,건성,각질,유니크(힙)


In [25]:
df2.columns

Index(['userID', 'user_tone', 'user_skintype', 'user_trouble', 'user_style'], dtype='object')

In [26]:
## DB연결후, 저장

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='1111',\
    db = 'beaulol', charset='utf8')
cur= conn.cursor()

cur.execute(""" 
            create table if not exists customer_type ( 
            userID VARCHAR(10) NOT NULL,
            user_tone VARCHAR(5) ,
            user_skintype VARCHAR(5) ,
            user_trouble VARCHAR(20) ,
            user_style VARCHAR(10) ,
            PRIMARY KEY(userID)
            )
            """)

for idx,row in df2.iterrows():
    try:
        sql = "INSERT INTO customer_type(userID, user_tone, user_skintype, user_trouble, user_style)\
            values(%s, %s, %s, %s, %s)"
        
        cur.execute(sql, (row['userID'],row['user_tone'],row['user_skintype'],row['user_trouble'], row['user_style']))
        
    except Exception as e:
        print("Error:", e)
        continue

conn.commit()
conn.close()

Error: (1062, "Duplicate entry 'M0001' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0002' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0003' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0004' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0005' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0006' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'M0007' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0008' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0009' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0010' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0011' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0012' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0013' for key 'customer_type.PRIMARY'")
Error: (1062, "Duplicate entry 'F0014' for key 'cus

3. 제거대상용매 DB 넣기

In [ ]:
import pymysql
import csv

conn = pymysql.connect(host='localhost', user='root', password='1111', db = 'beaulol', charset='utf8')
cur = conn.cursor()

f = open('./data/02.데이터수집_02DB전처리_용매_solvent_EK.csv', 'r', encoding='utf-8')

csvReader = list(csv.reader(f))
cur.execute("CREATE TABLE IF NOT EXISTS solvent (ingredient_eng VARCHAR(255), ingredient_kor VARCHAR(255))")

for row in csvReader[1:]:
    try:
        eng_name = row[0]
        kor_name = row[1]
        sql = "INSERT INTO solvent (ingredient_eng, ingredient_kor) VALUES (%s, %s);"
        cur.execute(sql, (eng_name, kor_name))
        
    except Exception as e:
            print("Error:", e)
            continue

cur.execute("insert into solvent values('WATER', '정제수')")    
f.close()
conn.commit()
conn.close()

4. 리뷰 DB 넣기

In [ ]:
import pandas as pd
csv_file ='./data/02.데이터수집_01크롤링_올리브영_리뷰_dokdo_review.csv'
df = pd.read_csv(csv_file)
df

,brand,prod_name,id,star_rate,review
0,라운드랩,라운드랩 1025 독도 로션,커비공주,4,한달차리뷰입니다!한달 넘게 쓴바로 저정도 썼는데요 대용량이라 짱짱합니다~딱 무난하고...
1,라운드랩,라운드랩 1025 독도 로션,러블리윤,5,초6 남아가 사용중이에요 ㅎ늘 키즈 로션 제품만 사용하다가유분기가 생기고나서부턴 트...
2,라운드랩,라운드랩 1025 독도 로션,바바바박,5,좋아요 정말 대용량 편해요 남편이 좋아해요 마음에 들어요 굿입니다 굿마음에쏘옥
3,라운드랩,라운드랩 1025 독도 로션,Alliveholy,5,계속 쟁여놓고 쓸정도로 최애 로션입니다.저때문에 저희가족 이거 다 써요.다 좋다는 ...
4,라운드랩,라운드랩 1025 독도 로션,하설희,5,예전에 써보고 좋아서 다시 독도 구매했어요독도 쓰다가 더심플 쓰려니깐 흡수도 약하고...
...,...,...,...,...,...
995,라운드랩,라운드랩 1025 독도 로션,mk****,5,"무난하게 잘 쓰고 있습니다. 트러블도 없고, 순합니다. 자주 사요"
996,라운드랩,라운드랩 1025 독도 로션,뫙망망,5,평소에쓰던 토너 앰플은 그대로고 로션만 바꿨는데 속당김 건조함이 없어요
997,라운드랩,라운드랩 1025 독도 로션,tl****,5,무겁지도 않고 가볍지도 않고 촉촉하게 사용 하는 로션 입니다
998,라운드랩,라운드랩 1025 독도 로션,kde****,5,독도스킨 쓰고 잇어서 로션도 같이 샀는데 데일리하게 바르기 좋아용


In [ ]:
conn = pymysql.connect(host='localhost', user='root', password='1111',\
    db = 'beaulol', charset='utf8')
cur= conn.cursor()

f=open('./data/02.데이터수집_01크롤링_올리브영_리뷰_dokdo_review.csv','r')

csvReader = list(csv.reader(f))
cur.execute("create table if not exists review_db (prd_brand VARCHAR(20), prd_name VARCHAR(40), userID VARCHAR(20),\
star_rate INT, review TEXT, PRIMARY KEY (userID))")


for data in csvReader[1:]:
    row1= data[0]
    row2= data[1]
    row3= data[2]
    row4= int(data[3])
    row5= data[4]
    
    print(row3,row4,type(row4))
    
    try:
        sql = """insert into review_db (prd_brand, prd_name, userID, star_rate, review) values(%s, %s, %s, %s, %s);"""
        cur.execute(sql,(row1,row2,row3,row4,row5))
    except:
        continue
    
f.close()
conn.commit()


conn.close()

커비공주 4 <class 'int'>
러블리윤 5 <class 'int'>
바바바박 5 <class 'int'>
Alliveholy 5 <class 'int'>
하설희 5 <class 'int'>
k202308140**** 5 <class 'int'>
보라빛보람 5 <class 'int'>
본예이 5 <class 'int'>
대방 5 <class 'int'>
카페라떼 5 <class 'int'>
학연아넌내마지막이야 5 <class 'int'>
나믿지 5 <class 'int'>
호호호아줌마 5 <class 'int'>
진난 5 <class 'int'>
H3 4 <class 'int'>
삼색이냥 5 <class 'int'>
올영세상 5 <class 'int'>
뚜34 5 <class 'int'>
윈탬 5 <class 'int'>
가나리가나 5 <class 'int'>
Dingincar 5 <class 'int'>
V 5 <class 'int'>
운수대톨 5 <class 'int'>
카페라떼 5 <class 'int'>
07서연 5 <class 'int'>
mzbz 5 <class 'int'>
히나다 5 <class 'int'>
dpfla**** 5 <class 'int'>
k202308140**** 5 <class 'int'>
꺄르르륵 4 <class 'int'>
artistyo**** 5 <class 'int'>
kjw**** 4 <class 'int'>
우항항항 5 <class 'int'>
예비공주 5 <class 'int'>
소로록 5 <class 'int'>
보라빛보람 5 <class 'int'>
꾸꾸망개 5 <class 'int'>
꾸꾸망개 5 <class 'int'>
Jessicaaa 5 <class 'int'>
skdip**** 4 <class 'int'>
한결825 5 <class 'int'>
V 5 <class 'int'>
ddd**** 5 <class 'int'>
yeojin**** 5 <class 'int'>
youngsik**** 5 <cl

5. 화장품 상품정보 DB 넣기

In [ ]:
def categorized(df,category):
    df['제품유형'] = f'{category}'
    return df

In [ ]:
import pandas as pd
cream = pd.read_csv('./data/02.데이터수집_01크롤링_올리브영_제품_크림.csv')
lotion = pd.read_csv('./data/02.데이터수집_01크롤링_올리브영_제품_로션.csv')
skin = pd.read_csv('./data/02.데이터수집_01크롤링_올리브영_제품_스킨.csv')
serum = pd.read_csv('./data/02.데이터수집_01크롤링_올리브영_제품_세럼.csv')

In [ ]:
serum = categorized(serum,'세럼')
cream = categorized(cream,'크림')
skin = categorized(skin,'스킨토너')
lotion = categorized(lotion,'로션')

In [ ]:
print(serum['제품유형'].head())
print(cream['제품유형'].head())
print(skin['제품유형'].head())
print(lotion['제품유형'].head())

0    세럼
1    세럼
2    세럼
3    세럼
4    세럼
Name: 제품유형, dtype: object
0    크림
1    크림
2    크림
3    크림
4    크림
Name: 제품유형, dtype: object
0    스킨토너
1    스킨토너
2    스킨토너
3    스킨토너
4    스킨토너
Name: 제품유형, dtype: object
0    로션
1    로션
2    로션
3    로션
4    로션
Name: 제품유형, dtype: object


In [ ]:
import pymysql

def update_prd_info(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df[['제품명','브랜드', '가격', '용량', '성분', '유통기한', '제품유형']]
    # Database connection
    conn = pymysql.connect(host='localhost', user='root', password='1111', db='beaulol', charset='utf8')

    with conn.cursor() as curs:
        # Create table if it does not exist
        curs.execute("""
        CREATE TABLE IF NOT EXISTS product_info (
            prd_name VARCHAR(500),
            prd_brand VARCHAR(50),
            prd_price VARCHAR(10),
            prd_capa TEXT,
            prd_ingredients TEXT,
            prd_exp TEXT,
            prd_category VARCHAR(10),
            PRIMARY KEY (prd_name)
        );
        """)

        # Prepare the SQL statement for inserting data
        sql = """
        REPLACE INTO product_info (prd_name, prd_brand, prd_price, prd_capa, prd_ingredients, prd_exp, prd_category)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """

        # Convert DataFrame to a list of tuples
        data_tuples = [tuple(x) for x in df.to_numpy()]

        # Execute the SQL statement in bulk
        curs.executemany(sql, data_tuples)

    # Commit changes and close connection
    conn.commit()
    conn.close()


In [ ]:
update_prd_info(cream)
update_prd_info(lotion)
update_prd_info(serum)
update_prd_info(skin)

In [ ]:
import pymysql

conn = pymysql.connect(host='localhost', user='root', password='1111', db='beaulol', charset='utf8')

with conn.cursor() as curs:
    sql = ("""
    select * from product_info;
    """)

    df = pd.read_sql(sql, conn)

# Commit changes and close connection
conn.commit()
conn.close()

print(df)

In [ ]:
df.to_csv('./data/02.데이터수집_02DB전처리_올리브영_제품_product_info.csv', index=True)

6. 고객 화장대 DB 넣기

In [ ]:
conn = pymysql.connect(host='localhost', user='root', password='1111', db = 'beaulol', charset='utf8')
cur= conn.cursor()

sql = """select * from customer_info"""
sql3 = """select * from product_info where prd_ingredients like '%,%'"""
customer_info = pd.read_sql(sql, conn)
product_info = pd.read_sql(sql3,conn)

C:\Users\SSO\AppData\Local\Temp\ipykernel_25388\3260332660.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_info = pd.read_sql(sql, conn)
C:\Users\SSO\AppData\Local\Temp\ipykernel_25388\3260332660.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_info = pd.read_sql(sql3,conn)


In [ ]:
customer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userID    9999 non-null   object
 1   name      9999 non-null   object
 2   address   9999 non-null   object
 3   email     9999 non-null   object
 4   phonenum  9999 non-null   object
 5   nickname  9999 non-null   object
 6   password  9999 non-null   object
 7   birth     9999 non-null   object
 8   sex       9999 non-null   object
dtypes: object(9)
memory usage: 703.2+ KB


In [ ]:
product_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prd_name         393 non-null    object
 1   prd_brand        393 non-null    object
 2   prd_price        393 non-null    object
 3   prd_capa         393 non-null    object
 4   prd_ingredients  393 non-null    object
 5   prd_exp          393 non-null    object
 6   prd_category     393 non-null    object
dtypes: object(7)
memory usage: 21.6+ KB


##### 랜덤뽑기 잘 되는지 확인

In [ ]:
import numpy as np
product = np.random.choice(list(product_info.loc[product_info['prd_category']=='세럼']['prd_name'].unique()),1)[0]
product

'리제덤365 모공탄력 캡슐세럼'

In [ ]:
product_info.loc[product_info['prd_name']==product]['prd_brand']

101    에스트라
Name: prd_brand, dtype: object

##### DB 저장

In [ ]:
conn = pymysql.connect(host='localhost', user='root', password='1111', db = 'beaulol', charset='utf8')
cur= conn.cursor()

create_sql = """ 
        create table if not exists customer_regis ( 
        regis_num VARCHAR(30) NOT NULL UNIQUE,
        regis_date DATE,
        userID VARCHAR(10) NOT NULL,
        prd_brand VARCHAR(50) NOT NULL, 
        prd_name VARCHAR(500) NOT NULL, 
        pay_date DATE, 
        pay_price VARCHAR(10),
        op_date DATE,
        PRIMARY KEY(regis_num)
        );
        """

cur.execute(create_sql)
        
conn.commit()
conn.close()

In [ ]:
conn = pymysql.connect(host='localhost', user='root', password='1111', db = 'beaulol', charset='utf8')
cur= conn.cursor()

insert_sql = "INSERT INTO customer_regis(regis_num, regis_date ,userID, prd_brand, prd_name) values(%s, %s, %s, %s, %s)"

for user in customer_info['userID']:
    for i in range(0,5):
        if i == 0 or i == 1:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='세럼']['prd_name'].unique()),1)[0]
        if i == 2:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='스킨토너']['prd_name'].unique()),1)[0]
        if i == 3:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='로션']['prd_name'].unique()),1)[0]
        if i == 4:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='크림']['prd_name'].unique()),1)[0]
        brand = product_info.loc[product_info['prd_name']==product]['prd_brand'].iloc[0]
        reginum = '240123' + "_" + str(user) + "_0" + str(i)
        
        cur.execute(insert_sql,(reginum,'2024-01-23',user,brand,product))
        
conn.commit()
conn.close()

##### DB에 저장한 데이터 데이터프레임으로 파일 저장

In [ ]:
conn = pymysql.connect(host='localhost', user='root', password='1111', db = 'beaulol', charset='utf8')
cur= conn.cursor()

from_sql = "select * from customer_regis"
dfs = pd.read_sql(from_sql,conn)
dfs

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_25386/1284692759.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs = pd.read_sql(from_sql,conn)


,regis_num,regis_date,userID,prd_brand,prd_name,pay_date,pay_price,op_date
0,240123_F0002_00,2024-01-23,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플,None,None,None
1,240123_F0002_01,2024-01-23,F0002,닥터지,에이클리어 스팟 포 페이스 세럼,None,None,None
2,240123_F0002_02,2024-01-23,F0002,라운드랩,1025 독도 토너,None,None,None
3,240123_F0002_03,2024-01-23,F0002,아벤느,트릭세라 로션,None,None,None
4,240123_F0002_04,2024-01-23,F0002,에스트라,아토베리어365 하이드로 수딩크림,None,None,None
...,...,...,...,...,...,...,...,...
49990,240123_M9994_00,2024-01-23,M9994,메디필,히알루론산 레이어 물톡스 앰플 더블 기획,None,None,None
49991,240123_M9994_01,2024-01-23,M9994,닐스야드 레머디스,레머디스 와일드 로즈 뷰티 세럼,None,None,None
49992,240123_M9994_02,2024-01-23,M9994,닥터지,더모이스처 배리어 D 리퀴드,None,None,None
49993,240123_M9994_03,2024-01-23,M9994,아크네스,모이스처 플루이드,None,None,None


In [27]:
import pandas as pd 
df = pd.read_csv('./data/02.데이터수집_03DB저장_고객등록화장품_샘플링 및 저장.csv')
df

,regis_num,regis_date,userID,prd_brand,prd_name,pay_date,pay_price,op_date
0,240123_F0002_00,2024-01-23,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플,NaN,NaN,NaN
1,240123_F0002_01,2024-01-23,F0002,닥터지,에이클리어 스팟 포 페이스 세럼,NaN,NaN,NaN
2,240123_F0002_02,2024-01-23,F0002,라운드랩,1025 독도 토너,NaN,NaN,NaN
3,240123_F0002_03,2024-01-23,F0002,아벤느,트릭세라 로션,NaN,NaN,NaN
4,240123_F0002_04,2024-01-23,F0002,에스트라,아토베리어365 하이드로 수딩크림,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
49990,240123_M9994_00,2024-01-23,M9994,메디필,히알루론산 레이어 물톡스 앰플 더블 기획,NaN,NaN,NaN
49991,240123_M9994_01,2024-01-23,M9994,닐스야드 레머디스,레머디스 와일드 로즈 뷰티 세럼,NaN,NaN,NaN
49992,240123_M9994_02,2024-01-23,M9994,닥터지,더모이스처 배리어 D 리퀴드,NaN,NaN,NaN
49993,240123_M9994_03,2024-01-23,M9994,아크네스,모이스처 플루이드,NaN,NaN,NaN
